In [1]:
# importando libs
import pandas as pd
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import os
from datetime import datetime
from datetime import timedelta
import yfinance as yf
import pickle
from prophet import Prophet
from prophet.serialize import model_to_json, model_from_json
from prophet import plot
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

/home/fabricio/Documentos/Projetos/tcc_predict_crypto/model_predict/.predict/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DAYS_PREDICT = 15

In [3]:
date_today = datetime.today().strftime("%Y-%m-%d")
data_init = "2014-01-01"

df_bnb = yf.download("BNB-USD", data_init, date_today)

df_bnb.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-11-09,2.05314,2.17423,1.89394,1.99077,1.99077,19192200
2017-11-10,2.00773,2.06947,1.64478,1.79684,1.79684,11155000
2017-11-11,1.78628,1.91775,1.61429,1.67047,1.67047,8178150
2017-11-12,1.66889,1.67280,1.46256,1.51969,1.51969,15298700
2017-11-13,1.52601,1.73502,1.51760,1.68662,1.68662,12238800


In [4]:
df_bnb.reset_index(inplace=True)
df_bnb.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
1768,2022-09-12,295.174103,299.176300,291.150696,293.658722,293.658722,998624528
1769,2022-09-13,293.649231,297.687714,275.968964,277.576935,277.576935,1136956066
1770,2022-09-14,277.413300,282.150269,274.830566,279.385864,279.385864,956413768
1771,2022-09-15,279.383057,280.246796,268.010223,270.806885,270.806885,1080164338
1772,2022-09-16,270.812256,276.263245,269.129517,274.654572,274.654572,803452434


In [5]:
df = df_bnb[["Date", "Adj Close"]]
df.rename(columns= { "Date": "ds", "Adj Close": "y" }, inplace=True)

In [6]:
df.tail()

,ds,y
1768,2022-09-12,293.658722
1769,2022-09-13,277.576935
1770,2022-09-14,279.385864
1771,2022-09-15,270.806885
1772,2022-09-16,274.654572


In [7]:
assert df.isnull().sum().sum() == 0
assert df['ds'].isnull().sum() == 0
assert df['y'].isnull().sum() == 0

In [8]:
df.dtypes

ds    datetime64[ns]
y            float64
dtype: object

In [9]:
model = Prophet(seasonality_mode="multiplicative")
model.fit(df)

00:32:43 - cmdstanpy - INFO - Chain [1] start processing
00:32:43 - cmdstanpy - INFO - Chain [1] done processing


In [10]:
df_future = model.make_future_dataframe(periods=DAYS_PREDICT)
df_future.tail()

,ds
1783,2022-09-27
1784,2022-09-28
1785,2022-09-29
1786,2022-09-30
1787,2022-10-01


In [11]:
# predict 
predict = model.predict(df_future)
predict

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,additive_terms,additive_terms_lower,additive_terms_upper,yhat
0,2017-11-09,6.726786,-25.165587,42.971790,6.726786,6.726786,0.360476,0.360476,0.360476,0.000584,0.000584,0.000584,0.359892,0.359892,0.359892,0.0,0.0,0.0,9.151633
1,2017-11-10,6.739698,-23.743657,40.594417,6.739698,6.739698,0.367930,0.367930,0.367930,-0.001165,-0.001165,-0.001165,0.369096,0.369096,0.369096,0.0,0.0,0.0,9.219438
2,2017-11-11,6.752610,-24.779923,45.207023,6.752610,6.752610,0.375245,0.375245,0.375245,-0.001985,-0.001985,-0.001985,0.377230,0.377230,0.377230,0.0,0.0,0.0,9.286491
3,2017-11-12,6.765521,-25.396828,42.673792,6.765521,6.765521,0.384993,0.384993,0.384993,0.000711,0.000711,0.000711,0.384283,0.384283,0.384283,0.0,0.0,0.0,9.370203
4,2017-11-13,6.778433,-23.824780,43.389262,6.778433,6.778433,0.389085,0.389085,0.389085,-0.001169,-0.001169,-0.001169,0.390254,0.390254,0.390254,0.0,0.0,0.0,9.415820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1783,2022-09-27,301.336252,222.327652,286.734872,301.336252,301.336252,-0.154796,-0.154796,-0.154796,0.000892,0.000892,0.000892,-0.155688,-0.155688,-0.155688,0.0,0.0,0.0,254.690484
1784,2022-09-28,300.905883,222.750503,290.692641,300.905883,300.905883,-0.148746,-0.148746,-0.148746,0.002134,0.002134,0.002134,-0.150880,-0.150880,-0.150880,0.0,0.0,0.0,256.147246
1785,2022-09-29,300.475514,222.423498,290.724929,300.475514,300.475514,-0.144896,-0.144896,-0.144896,0.000584,0.000584,0.000584,-0.145480,-0.145480,-0.145480,0.0,0.0,0.0,256.937876
1786,2022-09-30,300.045145,224.821926,291.822665,300.045145,300.045145,-0.140724,-0.140724,-0.140724,-0.001165,-0.001165,-0.001165,-0.139558,-0.139558,-0.139558,0.0,0.0,0.0,257.821691


In [12]:
predict[["ds", "yhat_lower", "yhat_upper", "yhat"]].tail()

,ds,yhat_lower,yhat_upper,yhat
1783,2022-09-27,222.327652,286.734872,254.690484
1784,2022-09-28,222.750503,290.692641,256.147246
1785,2022-09-29,222.423498,290.724929,256.937876
1786,2022-09-30,224.821926,291.822665,257.821691
1787,2022-10-01,224.670914,293.629224,259.116970


In [13]:
# Salvar modelo
with open('model_bnb_usd.pckl', 'wb') as fout:
    pickle.dump(model, fout)